In [ ]:
import xarray as xr
import numpy  as np
import pandas as pd
import matplotlib.pyplot as plt
import skill_metrics as sm
# import proplot as pplt

# 泰勒图绘制

2021.08.15

计算年际变化（时间序列）的taylor 图

2021.08.16

继续使用skillmetics包进行

将数据处理为字典，便于按照某个特征进行索引

泰勒图上的每个小点会单独绘制，按照特征给予不同的风格

## 数据读入

读入分区之后的数据

In [ ]:
# get partial mask_res collect into a dict
partial_list = ["NE", "NC", "YZ", "SC",'SW','NWC']
path_in = "/raid52/yycheng/MPAS/REFERENCE/TEMP_DATA_large/partial_t2m"
file_ref = "mask_sel_CN05.1_Tm_1961_2018_daily_025x025.nc"
file_vr = "mask_mean_t2m_98-17_VR.nc"
file_rcm = "mask_mean_t2m_98-17_RCM.nc"

var_partial = {}
for iregion in partial_list:
    path_in_partial = path_in + "/" + iregion + "/ordata/" # refine path
    var_partial[iregion] = {}
    var_partial[iregion]['obs'] = xr.open_dataset(path_in_partial + file_ref)['tm']
    var_partial[iregion]['vr'] = xr.open_dataset(path_in_partial + file_vr)['t2m'] - 273.15
    var_partial[iregion]['rcm'] = xr.open_dataset(path_in_partial + file_rcm)['t2m'] - 273.15
    # print(path_in_partial)
    # partial_res[i] = taylor_count(path_in_partial, file_ref, "pre", file_vr, "precip_MPAS", file_rcm, "precip_MPAS")

    # change coords
    var_list = ['obs', 'vr', 'rcm']
    for i in var_list:
        rename_dict = dict(zip(var_partial[iregion][i].coords.keys(), var_partial[iregion]['obs'].coords.keys()))
    #     # show converting coords
        for rename_i in rename_dict:
            print(rename_i + " -----converting to----- " + rename_dict[rename_i])

        var_partial[iregion][i] = var_partial[iregion][i].rename(rename_dict)
        var_partial[iregion][i]._coords = var_partial[iregion]['obs']._coords
        var_partial[iregion][i] = var_partial[iregion][i].rename(i)

## 计算年际时间序列 var_partial_ts

In [ ]:
time_sel = {}
time_sel['mjja'] = var_partial['NC']['obs'].time.dt.month.isin([5,6,7,8])

In [ ]:
# 计算年际时间序列：
var_partial_ts = {}
for iregion in partial_list:
    var_partial_ts[iregion] = {}
    for imod in ['obs','vr','rcm']:
        var_partial_ts[iregion][imod] = {}
        for iseason in ['mjja']:
            var_partial_ts[iregion][imod][iseason] = var_partial[iregion][imod].isel(time = time_sel[iseason]).mean(dim = ['lat','lon'])
            var_partial_ts[iregion][imod][iseason] = \
                var_partial_ts[iregion][imod][iseason].groupby(var_partial_ts[iregion][imod][iseason].time.dt.year).mean()

## 计算泰勒图诸要素，并整理进行绘图

In [ ]:
# 计算泰勒图诸要素 到字典taylor_ts中
taylor_ts = {}
for iregion in partial_list:
    taylor_ts[iregion] = {}
    for imod in ['obs','vr','rcm']:
        taylor_ts[iregion][imod] = {}
        for iseason in ['mjja']:
            taylor_ts[iregion][imod][iseason] = {}
            temp_obs = var_partial_ts[iregion]['obs'][iseason].values
            temp_mod = var_partial_ts[iregion][imod][iseason].values
            taylor_ts[iregion][imod][iseason] = sm.taylor_statistics(temp_mod, temp_obs)
            # normalized
            taylor_ts[iregion][imod][iseason]['crmsd'] = taylor_ts[iregion][imod][iseason]['crmsd'] / taylor_ts[iregion][imod][iseason]['sdev'][0]
            taylor_ts[iregion][imod][iseason]['sdev'] = taylor_ts[iregion][imod][iseason]['sdev'] / taylor_ts[iregion][imod][iseason]['sdev'][0]

### 从metrics中提取并且制表，用于和taylor定量化地分析

In [ ]:
# 计算泰勒图诸要素 到字典taylor_ts中
taylor_table = []
for iregion in partial_list:
    for imod in ['vr','rcm']:
        for iseason in ['mjja']:
            # 读入表格dict，去掉多余的观测
            taylor_table_temp = taylor_ts[iregion][imod][iseason].copy()
            for item in taylor_table_temp:
                taylor_table_temp[item] = taylor_table_temp[item][1] 
            # 转化为表格
            taylor_table_df = pd.DataFrame.from_dict(taylor_table_temp, orient='index', columns=['values'])
            taylor_table_df['season'] = iseason
            taylor_table_df['model'] = imod
            taylor_table_df['region'] = iregion
            # 整理到 list 中
            taylor_table.append(taylor_table_df)
# 合并list中的所有pd.dataframe
taylor_table = pd.concat(taylor_table)
# taylor_table
taylor_table.to_csv("./output_table/t2m_interannual_metrics.2022.01.30.csv")

# # 检查两个季节做差的状况
# taylor_table_oneseason = pd.DataFrame()
# taylor_table_oneseason['model'] = taylor_table.loc[taylor_table['season']=='jja']['model']
# taylor_table_oneseason['region'] = taylor_table.loc[taylor_table['season']=='jja']['region']
# taylor_table_oneseason['minus_values'] = taylor_table.loc[taylor_table['season']=='jja']['values'] - taylor_table.loc[taylor_table['season']=='am']['values']
# taylor_table_oneseason.to_csv("./output_table/t2m_interannual_metrics_oneseaon.csv")

# # 检查两个模式的状况
# taylor_table_modeldiff = pd.DataFrame()
# taylor_table_modeldiff['season'] = taylor_table.loc[taylor_table['model']=='rcm']['season']
# taylor_table_modeldiff['region'] = taylor_table.loc[taylor_table['model']=='rcm']['region']
# taylor_table_modeldiff['minus_values'] = taylor_table.loc[taylor_table['model']=='rcm']['values'] - taylor_table.loc[taylor_table['model']=='vr']['values']
# taylor_table_modeldiff.to_csv("./output_table/t2m_interannual_metrics_modeldiff.csv")

## PLOT
逐个点的绘制泰勒图，单独为每个点的属性设置相应的特征

In [ ]:
import skill_metrics as sm
import importlib

# fig, axs = pplt.subplots(ncols=1 ,nrows=1)
fig = plt.figure(figsize=(8, 8), dpi=600)

first_plot = False # 用作后续overlay设置

# 创建风格的dict,之后传入每个单独的点来进行绘制
# # Define markers
# kind = ['+','o','x','s','d','^','v','p','h','*']
# colorm = ['b','r','g','c','m','y','k']
imod_style_color = {'vr':"r", 'rcm':'b'}
iregion_style_color = {"NE":"m", "NC":"y", "YZ":"c", "SC":"b",'SW':"g",'NWC':"tab:gray"}
iseason_style_marker = {"mjja":'v'}

combine_mod_season_marker = {'vr mjja':"o","rcm mjja":"d"}

for iregion in partial_list:
    for imod in ['vr','rcm']:
        for iseason in ['mjja']:
            # 获取每个点的泰勒图诸要素
            sdev_plot = taylor_ts[iregion][imod][iseason]['sdev']
            crmsd_plot = taylor_ts[iregion][imod][iseason]['crmsd']
            ccoef_plot = taylor_ts[iregion][imod][iseason]['ccoef']

            sm.taylor_diagram(sdev_plot,crmsd_plot,ccoef_plot, 
                #----- pannel setting -----
                numberPanels = 2,
                #----- mark setting -----
                MarkerDisplayed = 'marker',
                # markerLabel = label2, 
                # numberPanels = 1,
                # markerLabel = ['obs','vr','rcm'],
                # markerLabel = ['obs', imod.upper()],
                # markerLabelColor = imod_style_color[imod],
                markerlegend = "off",
                markerColor = iregion_style_color[iregion],
                markerSize = 6,
                alpha = .5, 
                # -----RNSD-----
                # tickRMS = np.arange(0,1.8,0.2),
                tickRMS = [0, 0.5,1., 1.5],
                tickRMSangle = 110, 
                colRMS = 'g', styleRMS = ':', widthRMS = 1.5,
                # tickCOR = lsCOR,
                showlabelsRMS = "on",
                titleRMS = "off",
                rmslabelformat = ':.1f',
                # -----COR-----
                colCOR = "k", 
                widthCOR = 1.0,
                # -----STD------
                tickSTD = [0, 0.5, 1. ,1.5 , 1.8], axismax = 1.8, 
                showlabelsSTD = 'on',
                # markersymbol = iseason_style_marker[iseason],
                markersymbol = combine_mod_season_marker[imod + " " + iseason],
                # -----marker reference-----
                styleOBS = '-', 
                colOBS = 'k',
                markerobs = 'o', 
                overlay = first_plot,
                titleOBS = 'observation',
                # ------ check -----
                # checkStats = True,
            )

            first_plot = True


# legend设置，颜色以及型号
# custom legend
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
legend_elements = []
for i in iregion_style_color.items():
    legend_elements.append( Patch(facecolor=i[1],label=i[0]))
for i in combine_mod_season_marker.items():
    legend_elements.append( Line2D([0], [0], marker=i[1], color='w', label=i[0].upper(),
                          markerfacecolor='k', markersize=6)) 

fig.legend(handles=legend_elements, loc='upper right', bbox_to_anchor=(.92,0.8),prop={'size': 6}, ncol = 2)
fig.suptitle("surface air temperature", y=0.8)
fig.text(0.025,.75,"(b)", fontsize = 16, fontweight = "bold", color = 'k')
#saving pics
plt.savefig("./output_pic/t2m_taylor_interannual_ts.2022.01.30.png",dpi=600, facecolor = 'white')